In [1]:
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sc.logging.print_versions()
sc.set_figure_params(facecolor="white", figsize=(8, 8))
sc.settings.verbosity = 3

In [ ]:
adata = sc.read_visium()

In [ ]:
adata.var_names_make_unique()
# mitochondrial genes, MT for human Mt for mice
adata.var["mt"] = adata.var_names.str.startswith("MT-")
# ribosomal genes
adata.var["ribo"] = adata.var_names.str.startswith(("RPS", "RPL"))
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt', 'ribo'], percent_top=None, log1p=False, inplace=True)
sc.pl.violin(
adata,
["n_genes_by_counts", "total_counts", "pct_counts_mt", "pct_counts_ribo"],
jitter=0.4,
multi_panel=True,
) 

adata = adata[adata.obs.pct_counts_mt <= 20]
adata = adata[adata.obs.pct_counts_ribo < 1]

sc.pp.filter_cells(adata, min_genes=500) 
sc.pp.filter_genes(adata, min_cells=3) 

sc.pl.violin(
adata,
["n_genes_by_counts", "total_counts", "pct_counts_mt", "pct_counts_ribo"],
jitter=0.4,
multi_panel=True,
) 

In [ ]:
sc.pp.normalize_total(adata, inplace=True)
sc.pp.log1p(adata)

In [ ]:
sc.pp.regress_out(adata, ['total_counts'])

In [ ]:
sc.pp.scale(adata, max_value=15)
sc.pp.pca(adata, svd_solver='arpack', n_comps = 15)
sc.pl.pca_variance_ratio(adata, log=True)

In [ ]:
key = "n53_pc10"
resolution = 0.2
leidan_key = f"leiden_res{resolution}_{key}"
sc.pp.neighbors(adata, n_neighbors=53, n_pcs=10, key_added=key)

# Plot UMAP
sc.tl.leiden(adata, flavor="igraph", resolution=resolution, key_added=leidan_key, neighbors_key=key)
sc.tl.umap(adata, neighbors_key=key)
sc.pl.umap(adata, legend_loc="on data", color=[leidan_key])

In [ ]:
plt.rcParams["figure.figsize"] = (8, 8)
sc.pl.spatial(adata, img_key="hires", color=["total_counts", "n_genes_by_counts"])